In [1]:
import pandas as pd
import numpy as np

## tokenizer and deleting punctuation

In [2]:
from corpy.morphodita import Tokenizer
tokenizer = Tokenizer("czech")
punc='[]{}()"„“!@#$%^=_&*-:/,.\''
def tokeny(clanek):
    cont=[]
    for word in tokenizer.tokenize(clanek):
        cont.append(word)
    nopunc = [char for char in cont if char not in punc]
    return nopunc

## lemmatizer

In [3]:
from corpy.morphodita import Tagger
tagger = Tagger("./czech-morfflex-pdt-161115.tagger")
def lemmatizer(slovo):
    tokens = list(tagger.tag(slovo))
    a=''.join(str(tokens))
    ind1=a.find('a=')
    ind2=a.find("'",ind1+3)
    return a[ind1+3:ind2]

## stopwords

In [4]:
filename = 'stopwords-cs.txt'
stopwords = np.loadtxt(filename, dtype=str,encoding="utf8")
def delete_stopwords(lematizovana_slova):
    return list(set(lematizovana_slova)-set(stopwords))

## load data

In [5]:
df=pd.ExcelFile('C:/Users/vacla/OneDrive/Dokumenty/Audiozprávy/article_archive_cleared.xls')

In [6]:
articles=df.parse('article_archive')
articles.head()

,title,link,summary,published,tags,category,credit,source
0,"Pokud chceme pandemii udržet pod kontrolou, je...",https://www.lidovky.cz/domov/pokud-chceme-pand...,Příznivý vývoj epidemie koronaviru v Česku lze...,2021-05-28 20:20:00,['Zprávy - Domov'],Zprávy z domova,ČTK,lidovky.cz
1,Hlavní manažer ANO pobíral neoprávněně plný za...,https://www.lidovky.cz/domov/hlavni-manazer-an...,Hlavní manažer hnutí ANO Jan Richter loni pobí...,2021-05-28 18:53:00,['Zprávy - Domov'],Zprávy z domova,MAFRA,lidovky.cz
2,"Vzpomeňme si, co udělaly bary loni v létě, mla...",https://www.lidovky.cz/domov/prilisne-rozvolne...,"Po zmírnění protiepidemických opatření, ke kte...",2021-05-28 18:03:00,['Zprávy - Domov'],Zprávy z domova,Petr Lundák MAFRA,lidovky.cz
3,Poslední „Berdychovec“ je za mřížemi. Policie ...,https://www.lidovky.cz/domov/policie-privezla-...,Čeští kriminalisté přivezli z USA Jaromíra Pro...,2021-05-28 18:03:00,['Zprávy - Domov'],Zprávy z domova,Policie ČR,lidovky.cz
4,Mezi Českem a sedmi zeměmi bude po první dávce...,https://www.lidovky.cz/domov/mezi-ceskem-a-sed...,"Ze Slovenska, Polska, Německa, Rakouska, Maďar...",2021-05-28 18:01:00,['Zprávy - Domov'],Zprávy z domova,ČTK,lidovky.cz


### chose what features u want to use

In [7]:
#test
articles=articles[['title','summary']]

### prepare data, merge rows and create dataset

In [8]:
ddd=articles
#ddd['published']=np.datetime_as_string(ddd['published'])
for type in ddd.columns:
    ddd[type]=ddd[type].astype(str)

In [10]:
ddd.head()

,title,summary
0,"Pokud chceme pandemii udržet pod kontrolou, je...",Příznivý vývoj epidemie koronaviru v Česku lze...
1,Hlavní manažer ANO pobíral neoprávněně plný za...,Hlavní manažer hnutí ANO Jan Richter loni pobí...
2,"Vzpomeňme si, co udělaly bary loni v létě, mla...","Po zmírnění protiepidemických opatření, ke kte..."
3,Poslední „Berdychovec“ je za mřížemi. Policie ...,Čeští kriminalisté přivezli z USA Jaromíra Pro...
4,Mezi Českem a sedmi zeměmi bude po první dávce...,"Ze Slovenska, Polska, Německa, Rakouska, Maďar..."


### skip next cell if dont want to use time

In [137]:
for i in range(ddd.shape[0]):
    i1=ddd['published'][i][0:4]
    i2=ddd['published'][i][5:7]
    i3=ddd['published'][i][8:10]
    i4=ddd['published'][i][11:13]
    i5=ddd['published'][i][14:16]
    ddd['published'][i]=str(i1 + ' '+i2+' ' +i3+' ' +i4+' ' +i5 )
    #h=[i1,i2,i3,i4,i5]
    #for j in range(len(h)):
    #    ch=' '.join(h[i])
    #    ddd['published'].append(g)

KeyError: 'published'

In [11]:
data=[]
for i in range(ddd.shape[0]):
    g=' '.join(ddd.loc[i])
    data.append(g)

In [12]:
%%time
databaze=[]
d=[]
for clanek in data:
    d=tokeny(clanek)
    d=delete_stopwords(d) #change 1
    e=[]
    for slovo in d:
        e.append(lemmatizer(slovo))
    #e=delete_stopwords(e)
    databaze.append(e)
        

Wall time: 10.7 s


### to shape of CountVectorizer

In [13]:
dataset=[]
for index in range(len(databaze)):
    store=' '.join(databaze[index])
    dataset.append(store)


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors=vectorizer.fit_transform(dataset)

In [102]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
res=cosine_similarity(train_x_vectors[1],train_x_vectors[0])

Wall time: 2.99 ms


In [128]:
articles.loc[0:10,['title','summary','published','source']]


,title,summary,published,source
0,"Pokud chceme pandemii udržet pod kontrolou, je...",Příznivý vývoj epidemie koronaviru v Česku lze...,2021 05 28 20 20,lidovky.cz
1,Hlavní manažer ANO pobíral neoprávněně plný za...,Hlavní manažer hnutí ANO Jan Richter loni pobí...,2021 05 28 18 53,lidovky.cz
2,"Vzpomeňme si, co udělaly bary loni v létě, mla...","Po zmírnění protiepidemických opatření, ke kte...",2021 05 28 18 03,lidovky.cz
3,Poslední „Berdychovec“ je za mřížemi. Policie ...,Čeští kriminalisté přivezli z USA Jaromíra Pro...,2021 05 28 18 03,lidovky.cz
4,Mezi Českem a sedmi zeměmi bude po první dávce...,"Ze Slovenska, Polska, Německa, Rakouska, Maďar...",2021 05 28 18 01,lidovky.cz
5,"Děti by se mohly očkovat od konce června, uved...",Očkování proti covidu-19 je důležité pro děti ...,2021 05 28 17 30,lidovky.cz
6,Volné cestování po první dávce očkování je pln...,Ačkoli se Česko odvolává na reciprocitu se sou...,2021 05 28 16 00,lidovky.cz
7,"Úplné rozvolnění, kvůli soudu vláda od pondělí...",Přestřižení posledních proticovidových provazů...,2021 05 28 14 24,lidovky.cz
8,"Hamáček na vládě předloží postup, jak odškodni...",Ministr vnitra Jan Hamáček (ČSSD) v pondělí vl...,2021 05 28 14 02,lidovky.cz
9,"Mrhají nadšením vědců, říká o chování státu vi...",Výzkumný ústav BIOCEV už tři měsíce sekvenuje ...,2021 05 28 14 00,lidovky.cz


In [151]:
articles.loc[446,['title','summary']]


title      Vakcína pro děti nad dvanáct let: Kdy mohou př...
summary    Zítra má Evropská léková agentura (EMA) doporu...
Name: 446, dtype: object

## duplicated articles
* 1 a 441
* 5 a 446

In [145]:
cosine_similarity(train_x_vectors[0],train_x_vectors[4])

array([[0.15308091]])

In [146]:
cosine_similarity(train_x_vectors[1],train_x_vectors[441])

array([[0.54352309]])

In [147]:
cosine_similarity(train_x_vectors[5],train_x_vectors[446])

array([[0.35816181]])

In [148]:
cosine_similarity(train_x_vectors[1],train_x_vectors[10])

array([[0.10006256]])

In [ ]:
0-31 #28 05 - 27 05

In [ ]:
443 a1